In [16]:
from pathlib import Path
import yaml
from yaml import Loader

In [17]:
PATH_EXP = "/neurospin/dico/tsanchez/Jean_zay_runs/03_hyperparameter_full_ukbio/2025-05-22/11-25-32_13527/.hydra/config.yaml"
with open(PATH_EXP, "r") as f:
    file = yaml.load(f, Loader=Loader)
print(file)
file.keys()

{'dataset_name': 'Ukbio_vermis_JZ', 'data_root': '/lustre/fsn1/projects/rech/tgu/uyf32do/Runs/03_hyperparameter_full_ukbio/Input/Ukbio', 'save_dir': '/lustre/fsn1/projects/rech/tgu/uyf32do/Runs/03_hyperparameter_full_ukbio/Output', 'in_shape': [1, 54, 120, 139], 'kl': 2, 'embedding_size': 10, 'depth': 3, 'n': 512, 'batch_size': 32, 'nb_epoch': 500, 'lr': 0.001, 'weights': [5, 1, 5], 'run_time': '${now:%H-%M-%S}', 'sweep_dir': '${save_dir}/${now:%Y-%m-%d}/${run_time}_${suffix_dir}', 'suffix_dir': 13527, 'exp_settings': 'kl=${kl}_n=${n}_weights=${weights}'}


dict_keys(['dataset_name', 'data_root', 'save_dir', 'in_shape', 'kl', 'embedding_size', 'depth', 'n', 'batch_size', 'nb_epoch', 'lr', 'weights', 'run_time', 'sweep_dir', 'suffix_dir', 'exp_settings'])

In [31]:
from pathlib import Path
import yaml
from yaml import Loader

def retrieve_setting(path_exp : Path):
    with open(path_exp, "r") as f:
        settings = yaml.load(f, Loader=Loader)
    return settings


In [32]:
import os
ROOT_EXP = Path("/neurospin/dico/tsanchez/Jean_zay_runs/03_hyperparameter_full_ukbio/2025-05-22")
runs = [item for item in os.listdir(ROOT_EXP) if os.path.isdir(ROOT_EXP/ item)]
runs

['11-28-56_16320',
 '15-33-50_24423',
 '15-23-01_8291',
 '15-34-00_10698',
 '11-25-32_13527',
 '15-21-50_12910',
 '15-40-01_23631',
 '15-39-52_30186',
 '15-37-51_13539',
 '11-28-56_3749',
 '11-28-56_24690',
 '15-39-52_11615',
 '15-36-01_25032',
 '11-50-42_7979',
 '15-56-01_13666',
 '15-36-52_14355',
 '15-39-51_6124',
 '15-56-00_9647',
 '11-28-56_22946',
 '15-51-59_24970',
 '15-44-01_24676',
 '15-36-50_1518',
 '11-28-57_27270',
 '15-33-02_23227',
 '15-11-02_21292',
 '15-35-01_17',
 '15-55-52_15248',
 '11-28-56_7042',
 '15-40-52_14851',
 '15-29-00_16525',
 '15-40-52_22200',
 '15-41-02_30337',
 '15-21-02_5723',
 '15-48-00_26877',
 '11-28-04_3216',
 '15-45-50_10705',
 '15-45-01_4490',
 '15-11-51_21421',
 '15-28-54_30800',
 '11-28-49_13363',
 '15-26-59_10616',
 '15-12-01_29146',
 '11-28-58_27178',
 '15-56-02_2698',
 '15-21-59_9082',
 '15-24-54_9393']

In [33]:
settings_dict = {}
for directory in runs : 
    settings_dict[directory] = retrieve_setting(ROOT_EXP / directory / ".hydra/config.yaml")


In [34]:
import pandas as pd
df_settings = pd.DataFrame(settings_dict.values(), index=settings_dict.keys())
df_settings = df_settings[["n", "lr", "kl","weights"]]

In [22]:
import numpy as np
dict_val_run = dict()
for run in runs: 
    count_wm = np.load(ROOT_EXP / run / "wm_count.npy")
    count_sulci = np.load(ROOT_EXP / run / "sulci_count.npy")
    epochs = np.load(ROOT_EXP / run / "epoch_val.npy")
    dict_val_run[run] = {
        "count_wm" : count_wm,
        "count_sulci" : count_sulci, 
        "epochs" : epochs, 
    }
df_val = pd.DataFrame(dict_val_run.values(), index=dict_val_run.keys())

df_exp = pd.merge(df_settings, df_val, left_index=True, right_index=True, how="outer")

In [15]:
from tbparse import SummaryReader
reader = SummaryReader(ROOT_EXP / runs[0] / "logs", pivot=True)
df = reader.scalars

In [23]:
def retrieve_log(run : str):
    log_dir = ROOT_EXP / run / "logs"
    return SummaryReader(log_dir, pivot= True)

def stat_non_zeros(run : str) :
    stat_dict = {}
    df_log = retrieve_log(run) 
    # Nb epochs :
    stat_dict["nb_epoch"] = len(df_log["step"])

    # Proportion of nonZeros equal to 0
    stat_dict["perc_nz"] = (df_log["nonZeros/train"] == 0).sum() / stat_dict["nb_epoch"]
    
 

In [40]:
df_settings.sort_index()

,n,lr,kl,weights
11-25-32_13527,512,0.0010,2,"[5, 1, 5]"
11-28-04_3216,1024,0.0010,2,"[4, 1, 4]"
11-28-49_13363,512,0.0001,4,"[5, 1, 5]"
11-28-56_16320,512,0.0005,4,"[5, 1, 5]"
11-28-56_22946,1024,0.0005,4,"[5, 1, 5]"
11-28-56_24690,1024,0.0005,4,"[4, 1, 4]"
11-28-56_3749,512,0.0001,4,"[4, 1, 4]"
11-28-56_7042,512,0.0005,4,"[4, 1, 4]"
11-28-57_27270,1024,0.0001,4,"[5, 1, 5]"
11-28-58_27178,1024,0.0010,2,"[5, 1, 5]"


In [41]:
for exp in df_settings.sort_index().itertuples():
    print(f"- {exp.Index} : kl={exp.kl}, n={exp.n}, lr={exp.lr}, weights={exp.weights}")

- 11-25-32_13527 : kl=2, n=512, lr=0.001, weights=[5, 1, 5]
- 11-28-04_3216 : kl=2, n=1024, lr=0.001, weights=[4, 1, 4]
- 11-28-49_13363 : kl=4, n=512, lr=0.0001, weights=[5, 1, 5]
- 11-28-56_16320 : kl=4, n=512, lr=0.0005, weights=[5, 1, 5]
- 11-28-56_22946 : kl=4, n=1024, lr=0.0005, weights=[5, 1, 5]
- 11-28-56_24690 : kl=4, n=1024, lr=0.0005, weights=[4, 1, 4]
- 11-28-56_3749 : kl=4, n=512, lr=0.0001, weights=[4, 1, 4]
- 11-28-56_7042 : kl=4, n=512, lr=0.0005, weights=[4, 1, 4]
- 11-28-57_27270 : kl=4, n=1024, lr=0.0001, weights=[5, 1, 5]
- 11-28-58_27178 : kl=2, n=1024, lr=0.001, weights=[5, 1, 5]
- 11-50-42_7979 : kl=4, n=512, lr=0.001, weights=[5, 1, 5]
- 15-11-02_21292 : kl=4, n=1024, lr=0.001, weights=[4, 1, 4]
- 15-11-51_21421 : kl=4, n=1024, lr=0.001, weights=[5, 1, 5]
- 15-12-01_29146 : kl=6, n=512, lr=0.0001, weights=[4, 1, 4]
- 15-21-02_5723 : kl=6, n=512, lr=0.0001, weights=[5, 1, 5]
- 15-21-50_12910 : kl=6, n=1024, lr=0.0001, weights=[4, 1, 4]
- 15-21-59_9082 : kl=6, n=1